## Display Mean References

Reads the hsi-captures with the reference plastic samples.
Performs SNV on a selected ROI of each plastic and then displays the mean
of said SNV-transformed ROI using plotly in a browser window.

In [1]:
import numpy as np
import pandas as pd
import spectral
import plotly.express as px
import plotly.io as pio

from data_handling import get_hsi_capture, get_2d_roi, get_mean_plot, snv_transform

In [2]:
pd.options.plotting.backend = "plotly"
root = r"D:\OneDriveFHNW\FHNW\EUT-P6bb-21HS-RS_M365 - General\captures\references"
RGB_BANDS = (60, 80, 170)
BANDS = np.round(np.linspace(900,1700,224),1);
BAND_LOW = 8
BAND_HIGH = 210
PLOT_LABELS = {"index": "Wavelength [nm]", "value": "reflectance [1]", "variable": "pixel"}
COLUMN_NAMES = BANDS[BAND_LOW:BAND_HIGH]

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

spectral.settings.envi_support_nonlowercase_params = True

In [3]:
capture_list = []
capture_list.append(get_hsi_capture("LDPE", root))
capture_list.append(get_hsi_capture("PC", root))
capture_list.append(get_hsi_capture("PET_ref", root))
capture_list.append(get_hsi_capture("PLA", root))
capture_list.append(get_hsi_capture("PP", root))
capture_list.append(get_hsi_capture("PS", root))

INFO:data_handling:	Data Source:   'D:\OneDriveFHNW\FHNW\EUT-P6bb-21HS-RS_M365 - General\captures\references\p6bb-rs_LDPE_referenz_2021-11-16_14-53-53\capture\REFLECTANCE_p6bb-rs_LDPE_referenz_2021-11-16_14-53-53.dat'
	# Rows:            858
	# Samples:         640
	# Bands:           224
	Interleave:        BIL
	Quantization:  16 bits
	Data format:    uint16
INFO:data_handling:	Data Source:   'D:\OneDriveFHNW\FHNW\EUT-P6bb-21HS-RS_M365 - General\captures\references\p6bb-rs_PC_referenz_2021-11-16_15-11-36\capture\REFLECTANCE_p6bb-rs_PC_referenz_2021-11-16_15-11-36.dat'
	# Rows:            858
	# Samples:         640
	# Bands:           224
	Interleave:        BIL
	Quantization:  16 bits
	Data format:    uint16
INFO:data_handling:	Data Source:   'D:\OneDriveFHNW\FHNW\EUT-P6bb-21HS-RS_M365 - General\captures\references\p6bb-rs_PET_referenz_2021-11-16_14-58-52\capture\REFLECTANCE_p6bb-rs_PET_referenz_2021-11-16_14-58-52.dat'
	# Rows:            858
	# Samples:         640
	# Bands:       

In [4]:
# Load all data into a dict

capture_dict = {}
for capture in capture_list:
    # Laod and flip data vertical axis
    data = np.flip(capture.load(), axis=1)
    capture_name = capture.filename.split("\\")[-1].split("_202")[0].split("REFLECTANCE_")[1].split("_")[1]
    # Save to dict with filename as key
    capture_dict[capture_name] = data

In [5]:
roi_dict = {}
roi_dict["LDPE"] = {"y_low" : 220, "y_high": 360, "x_low" : 250, "x_high": 400}
roi_dict["PC"] = {"y_low" : 300, "y_high": 420, "x_low" : 260, "x_high": 400}
roi_dict["PET"] = {"y_low" : 220, "y_high": 340, "x_low" : 260, "x_high": 400}
roi_dict["PLA"] = {"y_low" : 300, "y_high": 550, "x_low" : 250, "x_high": 500}
roi_dict["PP"] = {"y_low" : 250, "y_high": 450, "x_low" : 300, "x_high": 450}
roi_dict["PS"] = {"y_low" : 360, "y_high": 500, "x_low" : 340, "x_high": 460}

roi_data_dict = {}
for name, data in capture_dict.items():
    roi_data_dict[name] = get_2d_roi(data,**roi_dict[name])

In [6]:
snv_data_dict = {}
for name, data in roi_data_dict.items():
    df = pd.DataFrame(data, columns = COLUMN_NAMES)
    np_snv = snv_transform(df)
    df_snv = pd.DataFrame(np_snv, columns=COLUMN_NAMES)
    snv_data_dict[name] = df_snv

In [7]:
mean_data_dict = {}
for name, data in snv_data_dict.items():
    mean_data_dict[name] = np.mean(snv_data_dict[name], axis=0)

In [8]:
df_mean = pd.DataFrame.from_dict(mean_data_dict, orient="index", columns=COLUMN_NAMES)

In [9]:
pio.renderers.default = "browser"
long_df_mean = pd.melt(df_mean, var_name="wavelength [nm]", ignore_index=False, value_name="reflectance [1]")
px.line(long_df_mean, x="wavelength [nm]", y="reflectance [1]", color=long_df_mean.index, title="Durchschnitts-Spektren der Referenzkunststoffe")

In [10]:
# # Export data to File
# mean_data_dict = {}
# for name, data in roi_data_dict.items():
#     mean_data_dict[name] = np.mean(roi_data_dict[name], axis=0)
# df_mean = pd.DataFrame.from_dict(mean_data_dict, orient="index", columns=COLUMN_NAMES)
# df_mean.to_csv(r"processed_data\mean_reference_plastics.csv", sep="\t")